# ChatQwen

This will help you get started with Qwen [chat models](../../concepts/chat_models.mdx). For detailed documentation of all ChatQwen features and configurations head to the [API reference](https://pypi.org/project/langchain-qwq/).

## Overview
### Integration details


| Class | Package | Local | Serializable | Package downloads | Package latest |
| :--- | :--- | :---: |  :---: | :---: | :---: |
| [ChatQwen](https://pypi.org/project/langchain-qwq/) | [langchain-qwq](https://pypi.org/project/langchain-qwq/) | ❌ | beta | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-qwq?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-qwq?style=flat-square&label=%20) |

### Model features
| [Tool calling](../../how_to/tool_calling.ipynb) | [Structured output](../../how_to/structured_output.ipynb) | JSON mode | [Image input](../../how_to/multimodal_inputs.ipynb) | Audio input | Video input | [Token-level streaming](../../how_to/chat_streaming.ipynb) | Native async | [Token usage](../../how_to/chat_token_usage_tracking.ipynb) | [Logprobs](../../how_to/logprobs.ipynb) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ |✅  | ❌ | ❌ | ✅ | ✅ | ✅ | ❌ | 


## Setup

To access Qwen models you'll need to create an Alibaba Cloud account, get an API key, and install the `langchain-qwq` integration package.

### Credentials

Head to [Alibaba's API Key page](https://account.alibabacloud.com/login/login.htm?oauth_callback=https%3A%2F%2Fbailian.console.alibabacloud.com%2F%3FapiKey%3D1&lang=en#/api-key) to sign up to Alibaba Cloud and generate an API key. Once you've done this set the `DASHSCOPE_API_KEY` environment variable:

In [1]:
import getpass
import os

if not os.getenv("DASHSCOPE_API_KEY"):
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter your Dashscope API key: ")

### Installation

The LangChain Qwen(by OpenAI API) integration lives in the `langchain-qwq` package:

In [ ]:
%pip install -qU langchain-qwq

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [3]:
from langchain_qwq import ChatQwen

llm = ChatQwen(
    model="qwen3-235b-a22b",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
    # enable_thinking=False, # disable thinking
    # other params...
)

## Invocation

In [4]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French."
        "Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime programmer.", additional_kwargs={'reasoning_content': 'Okay, the user wants me to translate "I love programming." into French. Let me start by breaking down the sentence. "I love" is straightforward; in French, that\'s "J\'aime". Now, "programming" can be a bit tricky. The direct translation might be "programmation", but sometimes people use "coder" in French to mean programming, especially in a more casual context. So, "J\'aime programmer" would be "I love to program". Alternatively, "la programmation" is the noun form, so "J\'aime la programmation" would mean "I love programming" as a general concept.\n\nI need to consider which is more appropriate. If the context is about the activity of programming itself, "programmer" as a verb might be better. But if it\'s about the field or concept, "programmation" as a noun could work. However, in common usage, "J\'aime programmer" sounds more natural and is often used. Also, "coder" is becoming more popular in French,

## Chaining

We can [chain](../../how_to/sequence.ipynb) our model with a prompt template like so:

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates"
            "{input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren.', additional_kwargs={'reasoning_content': 'Okay, the user wants me to translate "I love programming." into German. Let me think. The sentence is straightforward. "I love" in German is usually "Ich liebe" when talking about loving a person, but for activities or things, sometimes "Ich mag" is used. Wait, no, actually "Ich liebe" can also be used for strong affection towards activities. For example, "Ich liebe Tanzen" means "I love dancing." So "programming" as an activity, "Programmieren" is the noun. So "Ich liebe Programmieren." Alternatively, "Ich liebe das Programmieren." But maybe the user prefers the shorter version without the article. Let me check. In German, both forms are correct. Sometimes using the article makes it clearer. Hmm. Also, "programmieren" is the verb, but in the noun form, it\'s "das Programmieren." So the correct translation would be "Ich liebe das Programmieren." But sometimes people drop the article in informal s

## Tool Calling
ChatQwQ supports tool calling API that lets you describe tools and their arguments, and have the model return a JSON object with a tool to invoke and the inputs to that tool.

### Use with `bind_tools`

In [6]:
from langchain_core.tools import tool
from langchain_qwq import ChatQwen


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatQwen(model="qwen3-235b-a22b")

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("What's 5 times forty two")

print(msg)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking "What\'s 5 times forty two". Let me see. I need to multiply 5 by 42. The function provided is called multiply, which takes two integers. So first_int would be 5 and second_int is 42. I should call the multiply function with these values. Let me double-check the parameters: both are required and should be integers. Yep, that\'s correct. So the tool call should have the name "multiply" and the arguments as first_int:5 and second_int:42. That should', 'tool_calls': [{'index': 0, 'id': 'call_e36584f044db4446a9947e', 'function': {'arguments': '{"first_int": 5, "second_int": 42}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen3-235b-a22b'} id='run--85e21d77-c5c3-4e2f-b505-b822ecd55a92-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_e36584f044db4446a9947e', 'type': 'tool_call'}] usage_metadata={'input_tokens'

## Image Input
ChatQwen can invoke the QwenVL visual model, for example:

In [ ]:
from langchain_qwq import ChatQwen
from langchain_core.messages import HumanMessage

model = ChatQwen(model="qwen2.5-vl-72b-instruct")

response = model.invoke(
    [
        HumanMessage(
            content=[
                {
                    "type": "image_url",
                    "image_url": {"url": "http:example.com/image.png"},
                },
                {"type": "text", "text": "What do you see in this image?"},
            ]
        )
    ]
)
print(response)


content='The image shows a cute puppy with a light brown and white coat. The puppy has a white stripe down the middle of its face, white paws, and is standing on a soft surface. Its mouth is open, showing its tongue, and it appears to be looking directly at the camera with an alert and happy expression. The background is softly blurred, focusing attention on the puppy.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 1298, 'total_tokens': 1376, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': None, 'rejected_prediction_tokens': None, 'text_tokens': 78}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': None, 'text_tokens': 16, 'image_tokens': 1282}}, 'model_name': 'qwen2.5-vl-72b-instruct', 'system_fingerprint': None, 'id': 'chatcmpl-4e732a8b-4bbc-9722-b4a0-596fc979c0e9', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--0eef

## API reference

For detailed documentation of all ChatQwen features and configurations head to the [API reference](https://pypi.org/project/langchain-qwq/)